# PNW0483 Web-Based Pollutant Load Model
## Author: Jamie Feldman
### Date: 5/9/2022

In [7]:
import pandas as pd
import os

In [2]:
data_path = '01-input-tables'
result_path = '02-result-tables'

In [21]:
land_uses = ['AGR','COM','FOR','OPS','PUB','RES','TRA']
pollutants = ['TSS','TP','NO3','NH3','PB','CU','ZN','BOD','E.COLI','GLYPHOSATE','2,4-D','CARBARYL','OIL&GREASE']
bmps = ['Nutrient Management Plan - Agriculture',
        'Nutrient Management Plan - Urban', 
        'Integrated Pest Management',
        'Incentive Program', 
        'Conservation Buffer', 
        'Conservation Cover',
        'Grass Border', 
        'Buffer/Filter Strip', 
        'Riparian Buffer',
        'Critical Area Planting', 
        'Streamside Management Area',
        'Water Quality Basins', 
        'Bioretention/Biofilters', 
        'Media Filter',
        'Impervious Area Reduction', 
        'Organic Farming',
        'Drinking Water Protection Zones', 
        'Emergency Response Plan',
        'No-Till or Reduced Till', 
        'Cover Crops']

# Define Functions

In [4]:
def process_scenario(ui_subbasins, ui_bmps, runoff = None):
    runoff = calculate_runoff()
    

In [5]:
def calculate_runoff(land_uses, areas, rainfall, coeffs):
    runoff = pd.DataFrame()
    for lu in land_uses:
        _runoff = (areas.assign(LU = lu)
               .rename(columns = {'LU':'Land Use',lu:'Area (ac)'})
               .loc[lambda df: df['Area (ac)']>0]
               [['HUC','Area (ac)','Land Use']]
              )
        runoff = pd.concat([runoff,_runoff], ignore_index=True)

    runoff = (runoff.set_index(['HUC','Land Use'])
                     .join(rainfall.set_index('HUC'))
                     .join(coeffs.set_index(['HUC','Land Use']))
                     .sort_index()
                     .assign(Runoff = lambda df: df['Area (ac)']*df['Precip (in/yr)']*df['RC']/12*43560)
                    )
    return(runoff)

In [6]:
def load_no_attenuation(vol_df, vol_col, load_col, p, emcs):
    p_emcs = emcs.transpose()[[p]]
    _load = (vol_df.reset_index()
             .join(p_emcs, on = 'Land Use')
             .assign(Load = lambda df: df[vol_col]*df[p]*28.317/453592.37)
             .set_index(['HUC','Land Use'])
             .rename(columns = {'Load':p + load_col})
             [[p + load_col]]
            )
    return(_load)

In [7]:
def load_with_attenuation(load_no_att, pollutants, att_removal):
    load = load_no_att[['Runoff']]
    for p in pollutants:
        p_removal = att_removal[[p]]
        _load = (load_no_att
                 .reset_index()
                 .join(p_removal, on = 'HUC')
                 .assign(Load = lambda df: df[p+' Load']*(1-df[p]))
                 .set_index(['HUC','Land Use'])
                 .rename(columns = {'Load':p+' Attenuated Load'})
                 [[p+' Attenuated Load']]
                )
        load = load.join(_load)
    return(load)

# Calculate Runoff

In [8]:
rainfall = pd.read_csv(os.path.join(data_path,'Subbasin_Rainfall.csv'))
rainfall.head()

,HUC,Precip (in/yr)
0,101,73
1,102,73
2,103,73
3,104,73
4,105,73


In [9]:
coeffs = pd.read_csv(os.path.join(data_path,'Subcatchment_RCs.csv')).rename(columns = {'Average of Runoff Coefficient':'RC'})
coeffs.head()

,HUC,Land Use,RC
0,101,FOR,0.0280
1,101,TRA,0.0315
2,102,FOR,0.0280
3,102,OPS,0.0380
4,102,TRA,0.0180


In [10]:
areas = pd.read_csv(os.path.join(data_path,'Subbasin_Landuses.csv')).rename(columns = {'Subcatchment':'HUC'})
areas.head()

,HUC,AGR,COM,FOR,OPS,PUB,RES,TRA
0,101,0.0,0.0,10195.269430,0.000000,0.0,0.0,15.720060
1,102,0.0,0.0,9629.437767,859.572463,0.0,0.0,66.802055
2,103,0.0,0.0,18018.669430,146.185640,0.0,0.0,101.926739
3,104,0.0,0.0,17162.618870,0.000000,0.0,0.0,0.368773
4,105,0.0,0.0,10354.583280,0.000000,0.0,0.0,33.601803


In [11]:
runoff = calculate_runoff(land_uses, areas, rainfall, coeffs)
runoff.head()

Area (ac)  Precip (in/yr)      RC        Runoff
HUC Land Use                                                    
101 FOR       10195.269430              73  0.0280  7.564604e+07
    TRA          15.720060              73  0.0315  1.312182e+05
102 FOR        9629.437767              73  0.0280  7.144773e+07
    OPS         859.572463              73  0.0380  8.655568e+06
    TRA          66.802055              73  0.0180  3.186338e+05

In [12]:
# runoff.to_csv(os.path.join(result_path,'Runoff_Results.csv'))

# Calcuate Pre-BMP Loads without Attenuation

In [18]:
emcs = (pd.read_csv(os.path.join(data_path,'Landuse_Pollutant_EMCs.csv'))
#         .rename(columns = {'Unnamed: 0':'Pollutant'})
        .set_index('Pollutant')
       )
emcs

,AGR,COM,FOR,OPS,PUB,RES,TRA
Pollutant,,,,,,,
TSS,66.000000,81.697319,66.0000,52.900000,79.239516,135.263889,150.923077
TP,0.082000,0.450892,0.0160,0.175000,0.273820,0.408288,0.347292
NO3,2.445000,0.680896,0.0230,0.400000,0.503273,0.676500,1.530000
NH3,0.114000,1.560827,0.0020,0.737500,0.981019,1.471086,1.714583
PB,0.013400,0.040423,0.0134,0.002622,0.024161,0.021326,0.055408
CU,0.008500,0.025947,0.0085,0.004200,0.012777,0.015033,0.032000
ZN,0.050000,0.165353,0.0500,0.024889,0.075277,0.100930,0.211400
BOD,6.470000,13.553194,4.6670,4.666667,8.277027,9.761905,14.863636
E.COLI,1340.000000,3247.000000,1000.0000,1000.000000,1679.000000,2926.000000,6002.000000


In [14]:
pre_bmp_no_att = runoff[['Runoff']]
for p in pollutants:
    _load = load_no_attenuation(runoff, 'Runoff',' Load', p, emcs)
    pre_bmp_no_att = pre_bmp_no_att.join(_load)
pre_bmp_no_att

Runoff      TSS Load      TP Load      NO3 Load  \
HUC Land Use                                                          
101 FOR       7.564604e+07  3.116820e+05    75.559262    108.616439   
    TRA       1.312182e+05  1.236321e+03     2.844920     12.533350   
102 FOR       7.144773e+07  2.943838e+05    71.365766    102.588288   
    OPS       8.655568e+06  2.858464e+04    94.561668    216.140956   
    TRA       3.186338e+05  3.002126e+03     6.908244     30.434400   
...                    ...           ...          ...           ...   
607 FOR       6.673477e+07  2.749651e+05    66.658210     95.821177   
    OPS       5.775720e+07  1.907407e+05   630.994611   1442.273396   
    PUB       5.641522e+06  2.790742e+04    96.436724    177.247942   
    RES       3.246312e+08  2.741282e+06  8274.429754  13710.067989   
    TRA       6.115632e+07  5.762069e+05  1325.919551   5841.363629   

                  NH3 Load     PB Load     CU Load      ZN Load  \
HUC Land Use                                                      
101 FOR           9.444908   63.280882   40.140858   236.122693   
    TRA          14.045407    0.453885    0.262135     1.731732   
102 FOR           8.920721   59.768829   37.913063   223.018018   
    OPS         398.509887    1.416924    2.269480    13.448771   
    TRA          34.106088    1.102157    0.636536     4.205119   
...                    ...         ...         ...          ...   
607 FOR           8.332276   55.826251   35.412174   208.306905   
    OPS        2659.191574    9.454903   15.143871    89.741456   
    PUB         345.505723    8.509122    4.499925    26.512013   
    RES       29813.281827  432.191563  304.670299  2045.459661   
    TRA        6546.081517  211.540181  122.172311   807.100831   

                   BOD Load   E.COLI Load  GLYPHOSATE Load  2,4-D Load  \
HUC Land Use                                                             
101 FOR        22039.692200  4.722454e+06         0.000000    0.000000   
    TRA          121.758926  4.916678e+04         0.000000    0.000000   
102 FOR        20816.501800  4.460360e+06         0.000000    0.000000   
    OPS         2521.644484  5.403524e+05         0.000000    0.000000   
    TRA          295.663957  1.193904e+05         0.000000    0.000000   
...                     ...           ...              ...         ...   
607 FOR        19443.366555  4.166138e+06         0.000000    0.000000   
    OPS        16826.522952  3.605683e+06         0.000000    0.000000   
    PUB         2915.091414  5.913281e+05         0.024653    0.024653   
    RES       197836.478915  5.929883e+07         2.026618    1.418632   
    TRA        56747.650217  2.291494e+07         0.000000    0.000000   

              CARBARYL Load  OIL&GREASE Load  
HUC Land Use                                  
101 FOR            0.000000     13629.001862  
    TRA            0.000000        79.161920  
102 FOR            0.000000     12872.599999  
    OPS            0.000000       450.293658  
    TRA            0.000000       192.226781  
...                     ...              ...  
607 FOR            0.000000     12023.474583  
    OPS            0.000000      3004.736240  
    PUB            0.024653      1739.222269  
    RES            1.418632     74529.895811  
    TRA            0.000000     36894.649662  

[110 rows x 14 columns]

In [15]:
# pre_bmp_no_att.to_csv(os.path.join(result_path,'Pre-BMP_Load_NoAttenuation.csv'))

pre_bmp_no_att_total = pre_bmp_no_att.reset_index().groupby(['HUC']).sum()
# pre_bmp_no_att_total.to_csv(os.path.join(result_path,'Pre-BMP_Load_NoAttenuation_SubbasinTotals.csv'))
pre_bmp_no_att_total.head()

,Runoff,TSS Load,TP Load,NO3 Load,NH3 Load,PB Load,CU Load,ZN Load,BOD Load,E.COLI Load,GLYPHOSATE Load,"2,4-D Load",CARBARYL Load,OIL&GREASE Load
HUC,,,,,,,,,,,,,,
101,7.577726e+07,312918.276661,78.404182,121.149789,23.490315,63.734767,40.402993,237.854426,22161.451126,4.771621e+06,0.0,0.0,0.0,13708.163781
102,8.042193e+07,325970.551482,172.835678,349.163644,441.536696,62.287910,40.819080,240.671908,23633.810240,5.120103e+06,0.0,0.0,0.0,13515.120437
103,1.359722e+08,562841.103414,166.279345,300.312518,166.092000,114.616005,72.806304,429.344686,40080.088645,8.716239e+06,0.0,0.0,0.0,24607.857815
104,1.182475e+08,487219.832975,118.144226,169.932938,14.931150,98.922808,62.749235,369.115408,34452.757019,7.382481e+06,0.0,0.0,0.0,21305.093607
105,7.155854e+07,295996.887076,75.988248,123.270959,32.257929,60.433618,38.291904,225.561968,20987.651683,4.535399e+06,0.0,0.0,0.0,12984.867165


# Calculate Pre-BMP Loads with Attenuation

In [16]:
att_removal = pd.read_csv(os.path.join(data_path,'Subbasin_Attenuation_Percent_Removal.csv')).set_index('Subbasin')
att_removal.head()

,TSS,TP,NO3,NH3,PB,CU,ZN,BOD,E.COLI,GLYPHOSATE,"2,4-D",CARBARYL,OIL&GREASE
Subbasin,,,,,,,,,,,,,
101,0.741,0.5187,0.5187,0.5187,0.48165,0.48165,0.2223,0.2223,0.2223,0.5928,0.2223,0.5928,0.62985
102,0.741,0.5187,0.5187,0.5187,0.48165,0.48165,0.2223,0.2223,0.2223,0.5928,0.2223,0.5928,0.62985
103,0.741,0.5187,0.5187,0.5187,0.48165,0.48165,0.2223,0.2223,0.2223,0.5928,0.2223,0.5928,0.62985
104,0.741,0.5187,0.5187,0.5187,0.48165,0.48165,0.2223,0.2223,0.2223,0.5928,0.2223,0.5928,0.62985
105,0.741,0.5187,0.5187,0.5187,0.48165,0.48165,0.2223,0.2223,0.2223,0.5928,0.2223,0.5928,0.62985


In [341]:
pre_bmp_load = load_with_attenuation(pre_bmp_no_att, pollutants, att_removal)
pre_bmp_load.head()

Runoff  TSS Attenuated Load  TP Attenuated Load  \
HUC Land Use                                                          
101 FOR       7.564604e+07         80725.626411           36.366673   
    TRA       1.312182e+05           320.207244            1.369260   
102 FOR       7.144773e+07         76245.399993           34.348343   
    OPS       8.655568e+06          7403.422121           45.512531   
    TRA       3.186338e+05           777.550720            3.324938   

              NO3 Attenuated Load  NH3 Attenuated Load  PB Attenuated Load  \
HUC Land Use                                                                 
101 FOR                 52.277092             4.545834           32.801645   
    TRA                  6.032301             6.760055            0.235271   
102 FOR                 49.375743             4.293543           30.981172   
    OPS                104.028642           191.802809            0.734463   
    TRA                 14.648077            16.415260            0.571303   

              CU Attenuated Load  ZN Attenuated Load  BOD Attenuated Load  \
HUC Land Use                                                                
101 FOR                20.807014          183.632619         17140.268624   
    TRA                 0.135878            1.346768            94.691917   
102 FOR                19.652236          173.441113         16188.993450   
    OPS                 1.176385           10.459109          1961.082915   
    TRA                 0.329949            3.270321           229.937859   

              E.COLI Attenuated Load  GLYPHOSATE Attenuated Load  \
HUC Land Use                                                       
101 FOR                 3.672652e+06                         0.0   
    TRA                 3.823700e+04                         0.0   
102 FOR                 3.468822e+06                         0.0   
    OPS                 4.202321e+05                         0.0   
    TRA                 9.284989e+04                         0.0   

              2,4-D Attenuated Load  CARBARYL Attenuated Load  \
HUC Land Use                                                    
101 FOR                         0.0                       0.0   
    TRA                         0.0                       0.0   
102 FOR                         0.0                       0.0   
    OPS                         0.0                       0.0   
    TRA                         0.0                       0.0   

              OIL&GREASE Attenuated Load  
HUC Land Use                              
101 FOR                      5044.775039  
    TRA                        29.301785  
102 FOR                      4764.792890  
    OPS                       166.676197  
    TRA                        71.152743

In [309]:
pre_bmp_load.to_csv(os.path.join(result_path,'Pre-BMP_Load_WithAttenuation.csv'))

pre_bmp_load_total = pre_bmp_load.reset_index().groupby(['HUC']).sum()
pre_bmp_load_total.to_csv(os.path.join(result_path,'Pre-BMP_Load_WithAttenuation_SubbasinTotals.csv'))
pre_bmp_load_total.head()

,Runoff,TSS Attenuated Load,TP Attenuated Load,NO3 Attenuated Load,NH3 Attenuated Load,PB Attenuated Load,CU Attenuated Load,ZN Attenuated Load,BOD Attenuated Load,E.COLI Attenuated Load,GLYPHOSATE Attenuated Load,"2,4-D Attenuated Load",CARBARYL Attenuated Load,OIL&GREASE Attenuated Load
HUC,,,,,,,,,,,,,,
101,7.577726e+07,81045.833655,37.735933,58.309393,11.305889,33.036916,20.942892,184.979387,17234.960541,3.710889e+06,0.0,0.0,0.0,5074.076824
102,8.042193e+07,84426.372834,83.185812,168.052462,212.511612,32.286938,21.158570,187.170543,18380.014224,3.981904e+06,0.0,0.0,0.0,5002.621830
103,1.359722e+08,145775.845784,80.030249,144.540415,79.940080,59.411206,37.739148,333.901362,31170.284939,6.778619e+06,0.0,0.0,0.0,9108.598570
104,1.182475e+08,126189.936741,56.862816,81.788723,7.186362,51.276637,32.526066,287.061053,26793.909134,5.741356e+06,0.0,0.0,0.0,7886.080398
105,7.155854e+07,76663.193753,36.573144,59.330313,15.525741,31.325766,19.848608,175.419543,16322.096714,3.527180e+06,0.0,0.0,0.0,4806.348581


# Calculate Post-BMP Loads without Attenuation

In [233]:
ui_subbasins = [606, 607]
ui_bmps = pd.read_csv(os.path.join(data_path,'User_Input_Example.csv')).set_index('Land Use')
ui_bmps

,Nutrient Management Plan - Agriculture,Nutrient Management Plan - Urban,Integrated Pest Management,Incentive Program,Conservation Buffer,Conservation Cover,Grass Border,Buffer/Filter Strip,Riparian Buffer,Critical Area Planting,Streamside Management Area,Water Quality Basins,Bioretention/Biofilters,Media Filter,Impervious Area Reduction,Organic Farming,Drinking Water Protection Zones,Emergency Response Plan,No-Till or Reduced Till,Cover Crops
Land Use,,,,,,,,,,,,,,,,,,,,
AGR,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0,NaN,0.0,0.0
COM,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,50.0,NaN,0,0.0,NaN,NaN
FOR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
OPS,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
PUB,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0,0.0,NaN,NaN
RES,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0,0.0,NaN,NaN
TRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,0.0,NaN,NaN


In [234]:
bmp_wq = pd.read_csv(os.path.join(data_path, 'BMP_WQ_Performance.csv')).set_index('BMP Name')
bmp_wq

,TSS,TP,NO3,NH3,CU,PB,ZN,BOD,GLYPHOSATE,"2,4-D",CARBARYL,OIL&GREASE,E.COLI
BMP Name,,,,,,,,,,,,,
Nutrient Management Plan - Agriculture,NaN,0.2000,0.2000,0.2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Nutrient Management Plan - Urban,NaN,0.2000,0.2000,0.2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Integrated Pest Management,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.20,0.20,0.20,NaN,NaN
Incentive Program,NaN,0.1000,0.1000,0.1000,NaN,NaN,NaN,0.1000,NaN,NaN,NaN,NaN,0.10
Conservation Buffer,16.3500,0.1830,0.4395,0.1061,0.0069,0.0020,0.0236,4.6080,0.20,0.20,0.20,5.5465,0.25
Conservation Cover,16.3500,0.1830,0.4395,0.1061,0.0069,0.0020,0.0236,4.6080,0.20,0.20,0.20,5.5465,0.25
Grass Border,16.3500,0.1830,0.4395,0.1061,0.0069,0.0020,0.0236,4.6080,0.20,0.20,0.20,5.5465,0.25
Buffer/Filter Strip,16.3500,0.1830,0.4395,0.1061,0.0069,0.0020,0.0236,4.6080,0.20,0.20,0.20,5.5465,0.25
Riparian Buffer,16.3500,0.1830,0.4395,0.1061,0.0069,0.0020,0.0236,4.6080,0.20,0.20,0.20,5.5465,0.25


In [235]:
bmp_wq_type = pd.read_csv(os.path.join(data_path, 'BMP_Performance_Type.csv')).set_index('BMP Name')
bmp_wq_type.head()

,TSS,TP,NO3,NH3,CU,PB,ZN,BOD,GLYPHOSATE,"2,4-D",CARBARYL,OIL&GREASE,E.COLI
BMP Name,,,,,,,,,,,,,
Nutrient Management Plan - Agriculture,NaN,PR,PR,PR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Nutrient Management Plan - Urban,NaN,PR,PR,PR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Integrated Pest Management,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PR,PR,PR,NaN,NaN
Incentive Program,NaN,PR,PR,PR,NaN,NaN,NaN,PR,NaN,NaN,NaN,NaN,PR
Conservation Buffer,EQ,EQ,EQ,EQ,EQ,EQ,EQ,EQ,PR,PR,PR,EQ,PR


In [236]:
bmp_hydr = pd.read_csv(os.path.join(data_path, 'BMP_Hydraulic_Performance.csv')).set_index('BMP Name')
bmp_hydr

,Percent Capture,Percent Reduction
BMP Name,,
Nutrient Management Plan - Agriculture,1.0,0.00
Nutrient Management Plan - Urban,1.0,0.00
Integrated Pest Management,1.0,0.00
Incentive Program,1.0,0.00
Conservation Buffer,0.9,0.00
Conservation Cover,0.9,0.00
Grass Border,0.9,0.00
Buffer/Filter Strip,0.9,0.00
Riparian Buffer,0.9,0.00


In [237]:
minimum_conc = pd.read_csv(os.path.join(data_path, 'Pollutant_Minimum_Concentrations.csv')).set_index('Pollutant')
minimum_conc.head()

,Unit,Minimum Concentration
Pollutant,,
TSS,mg/L,8.100
TP,mg/L,0.090
NO3,mg/L,0.230
NH3,mg/L,0.030
PB,mg/L,0.001


In [286]:
total_bmp_load = runoff[['Runoff']].reset_index() #Create dataframe to track overall load from BMPs
total_bmp_load['Effluent Volume'] = 0
total_bmp_load['Volume Reduction'] = 0

for pol in pollutants:
    total_bmp_load[pol+' Effluent Load'] = 0
    
for bmp in bmps: #Iterate through BMPs
    bmp_load = runoff[['Runoff']].reset_index() #Create dataframe to track load from each BMP type
    
    for pol in pollutants: #Iterate through pollutants
        bmp_load['Volume to BMP'] = 0
        bmp_load['Volume Captured'] = 0
        bmp_load['Volume Bypassed'] = 0
        bmp_load['Volume Reduction'] = 0
        bmp_load['Effluent Volume'] = 0
        bmp_load[pol+' Effluent Load'] = 0
    
        for i, subcatch in bmp_load.iterrows(): #Iterate through model subcatchments (subbasin + land use)
            subbasin = subcatch['HUC']

            if subbasin in ui_subbasins: #If subbasin selected by user, continue with calcs 
                lu = subcatch['Land Use']
                total_percent_implemented = ui_bmps.loc[lu].sum() 

                if total_percent_implemented > 0: #If any BMPs relevant to land use are implemented, continue with calcs
                    bmp_percent_implemented = ui_bmps.loc[lu][bmp]/100

                    if bmp_percent_implemented > 0: #If BMP relevant to land use, continue with calcs
                        volume_to_bmp = subcatch['Runoff']*bmp_percent_implemented
                        capture_efficiency = bmp_hydr.loc[bmp]['Percent Capture']
                        retention_efficiency = bmp_hydr.loc[bmp]['Percent Reduction']
                        captured_volume = volume_to_bmp*capture_efficiency
                        bypassed_volume = volume_to_bmp - captured_volume
                        volume_retained = captured_volume*retention_efficiency 
                        effluent_volume = captured_volume-volume_retained
#                         volume_retained = subcatch['Runoff']*retention_efficiency #This is wrong but matches Excel

                        bmp_load.loc[i,'Volume to BMP'] = volume_to_bmp
                        bmp_load.loc[i,'Volume Captured'] = captured_volume
                        bmp_load.loc[i,'Volume Bypassed'] = bypassed_volume
                        bmp_load.loc[i,'Volume Reduction'] = volume_retained
                        bmp_load.loc[i,'Effluent Volume'] = effluent_volume
        
                        total_bmp_load.loc[i,'Effluent Volume'] = effluent_volume
                        total_bmp_load.loc[i,'Volume Reduction'] = volume_retained
                        
                        bmp_pol_performance = bmp_wq.loc[bmp][pol]
                        
                        if bmp_pol_performance > 0: #If BMP relevant to pollutant, continue with calcs
                            influent_wq = emcs.loc[pol][lu]
                            min_wq = minimum_conc.loc[pol]['Minimum Concentration']
                            
                            if influent_wq > min_wq: #If influent conc. higher than minimum, continue with calcs
                                bmp_performance_type = bmp_wq_type.loc[bmp][pol]
                                
                                if bmp_performance_type == 'PR':
                                    if influent_wq*(1-bmp_pol_performance) > min_wq:
                                        effluent_wq = influent_wq*(1-bmp_pol_performance)
                                    else:
                                        effluent_wq = min_wq
                                
                                if bmp_performance_type == 'EQ':
                                    if bmp_pol_performance < influent_wq:
                                        effluent_wq = bmp_pol_performance
                                    else: 
                                        effluent_wq = influent_wq
                                
                                effluent_load = effluent_wq*effluent_volume*28.317/453592.37
                            else:
                                effluent_load = influent_wq*effluent_volume*28.317/453592.37
                                
                            bmp_load.loc[i,pol+' Effluent Load'] = effluent_load
#                             print(bmp + ', ' + pol + ', ' + str(i) + ', ' + str(effluent_load))
                            old_effluent_load = total_bmp_load.loc[i,pol+' Effluent Load']
                            total_bmp_load.loc[i,pol+' Effluent Load'] = old_effluent_load + effluent_load
                        
    bmp_load = bmp_load.set_index(['HUC','Land Use'])
    bmp_load.to_csv(os.path.join(result_path,'BMP_Results_'+bmp.replace('/','-')+'.csv'))
    
total_bmp_load = total_bmp_load.set_index(['HUC','Land Use'])    
total_bmp_load.to_csv(os.path.join(result_path,'BMP_Results_All.csv'))

Impervious Area Reduction, TSS, 98, 178.324033920536
Impervious Area Reduction, TSS, 104, 36184.83396293139
Impervious Area Reduction, TP, 98, 6.4403340980672015
Impervious Area Reduction, TP, 104, 1306.8480724713447
Impervious Area Reduction, NO3, 98, 5.0682724807254695
Impervious Area Reduction, NO3, 104, 1028.434553447064
Impervious Area Reduction, NH3, 98, 0.6992237088376136
Impervious Area Reduction, NH3, 104, 141.88381257968172
Impervious Area Reduction, PB, 98, 0.02418698363903695
Impervious Area Reduction, PB, 104, 4.907930623825468
Impervious Area Reduction, CU, 98, 0.22427930283470623
Impervious Area Reduction, CU, 104, 45.5099021481998
Impervious Area Reduction, ZN, 98, 0.44196215558603874
Impervious Area Reduction, ZN, 104, 89.68127776262901
Impervious Area Reduction, BOD, 98, 54.48667768866688
Impervious Area Reduction, BOD, 104, 11056.229168945007
Impervious Area Reduction, E.COLI, 98, 53546.68355178612
Impervious Area Reduction, E.COLI, 104, 10865489.1378827
Impervious A

In [289]:
post_bmp_no_att = total_bmp_load[['Runoff','Effluent Volume','Volume Reduction']]
post_bmp_no_att['Untreated Volume'] = (post_bmp_no_att['Runoff'] 
                                       - post_bmp_no_att['Effluent Volume'] 
                                       - post_bmp_no_att['Volume Reduction']
                                      )
for p in pollutants:
    p_emcs = emcs.transpose()[[p]]
    no_bmp_load = (post_bmp_no_att.reset_index()
             .join(p_emcs, on = 'Land Use')
             .assign(Load = lambda df: df['Untreated Volume']*df[p]*28.317/453592.37)
             .set_index(['HUC','Land Use'])
             .rename(columns = {'Load':p+' No BMP Load'})
             [[p+' No BMP Load']]
            )
    all_bmp_load = (total_bmp_load
                    [[p+' Effluent Load']]
                   )
    _post_bmp_no_att = no_bmp_load.join(all_bmp_load)
    _post_bmp_no_att[p+' Load'] = _post_bmp_no_att[p+' No BMP Load'] + _post_bmp_no_att[p+' Effluent Load']
    post_bmp_no_att = post_bmp_no_att.join(_post_bmp_no_att[p+' Load'])

post_bmp_no_att.to_csv(os.path.join(result_path,'Post-BMP_Load_NoAttenuation.csv'))
post_bmp_no_att.tail(10)

C:\Users\jfeldman\AppData\Local\Temp\ipykernel_31440\1599058082.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_bmp_no_att['Untreated Volume'] = (post_bmp_no_att['Runoff']


Runoff  Effluent Volume  Volume Reduction  \
HUC Land Use                                                    
606 OPS       1.439002e+07     0.000000e+00      0.000000e+00   
    RES       2.009169e+07     0.000000e+00      0.000000e+00   
    TRA       9.255939e+06     0.000000e+00      0.000000e+00   
607 AGR       4.674467e+08     0.000000e+00      0.000000e+00   
    COM       1.868499e+08     7.147007e+07      1.261237e+07   
    FOR       6.673477e+07     0.000000e+00      0.000000e+00   
    OPS       5.775720e+07     0.000000e+00      0.000000e+00   
    PUB       5.641522e+06     0.000000e+00      0.000000e+00   
    RES       3.246312e+08     0.000000e+00      0.000000e+00   
    TRA       6.115632e+07     0.000000e+00      0.000000e+00   

              Untreated Volume      TSS Load      TP Load      NO3 Load  \
HUC Land Use                                                              
606 OPS           1.439002e+07  4.752241e+04   157.210250    359.337714   
    RES           2.009169e+07  1.696601e+05   512.111101    848.527115   
    TRA           9.255939e+06  8.720826e+04   200.676426    884.083787   
607 AGR           4.674467e+08  1.926006e+06  2392.915962  71349.750320   
    COM           1.027674e+08  5.603218e+05  4199.588932   5396.789825   
    FOR           6.673477e+07  2.749651e+05    66.658210     95.821177   
    OPS           5.775720e+07  1.907407e+05   630.994611   1442.273396   
    PUB           5.641522e+06  2.790742e+04    96.436724    177.247942   
    RES           3.246312e+08  2.741282e+06  8274.429754  13710.067989   
    TRA           6.115632e+07  5.762069e+05  1325.919551   5841.363629   

                  NH3 Load     PB Load     CU Load      ZN Load  \
HUC Land Use                                                      
606 OPS         662.528911    2.355658    3.773046    22.358791   
    RES        1845.167948   26.748683   18.856289   126.595141   
    TRA         990.742044   32.016367   18.490641   122.153799   
607 AGR        3326.736825  391.037486  248.046167  1459.095099   
    COM       10155.516733  264.248746  211.973070  1150.517195   
    FOR           8.332276   55.826251   35.412174   208.306905   
    OPS        2659.191574    9.454903   15.143871    89.741456   
    PUB         345.505723    8.509122    4.499925    26.512013   
    RES       29813.281827  432.191563  304.670299  2045.459661   
    TRA        6546.081517  211.540181  122.172311   807.100831   

                   BOD Load   E.COLI Load  GLYPHOSATE Load  2,4-D Load  \
HUC Land Use                                                             
606 OPS         4192.273333  8.983443e+05         0.000000    0.000000   
    RES        12244.258512  3.670052e+06         0.125429    0.087800   
    TRA         8588.692762  3.468151e+06         0.000000    0.000000   
607 AGR       188806.905755  3.910375e+07        12.022944   12.022944   
    COM        98008.043604  3.169693e+07         0.698950    0.698950   
    FOR        19443.366555  4.166138e+06         0.000000    0.000000   
    OPS        16826.522952  3.605683e+06         0.000000    0.000000   
    PUB         2915.091414  5.913281e+05         0.024653    0.024653   
    RES       197836.478915  5.929883e+07         2.026618    1.418632   
    TRA        56747.650217  2.291494e+07         0.000000    0.000000   

              CARBARYL Load  OIL&GREASE Load  
HUC Land Use                                  
606 OPS            0.000000       748.620238  
    RES            0.087800      4612.715088  
    TRA            0.000000      5583.963553  
607 AGR            2.918190     84218.969089  
    COM            0.698950     49585.229164  
    FOR            0.000000     12023.474583  
    OPS            0.000000      3004.736240  
    PUB            0.024653      1739.222269  
    RES            1.418632     74529.895811  
    TRA            0.000000     36894.649662

In [290]:
post_bmp_no_att_total = post_bmp_no_att.reset_index().groupby(['HUC']).sum()
post_bmp_no_att_total.to_csv(os.path.join(result_path,'Post-BMP_Load_NoAttenuation_SubbasinTotals.csv'))
post_bmp_no_att_total.tail(10)

,Runoff,Effluent Volume,Volume Reduction,Untreated Volume,TSS Load,TP Load,NO3 Load,NH3 Load,PB Load,CU Load,ZN Load,BOD Load,E.COLI Load,GLYPHOSATE Load,"2,4-D Load",CARBARYL Load,OIL&GREASE Load
HUC,,,,,,,,,,,,,,,,,
501,1.311734e+08,0.000000e+00,0.000000e+00,1.311734e+08,5.417455e+05,170.467634,310.969425,206.104188,109.657812,69.848284,411.836526,38628.926454,8.390665e+06,0.000000,0.000000,0.000000,23569.868230
502,1.821886e+08,0.000000e+00,0.000000e+00,1.821886e+08,7.437310e+05,620.485679,9113.425690,1407.465789,144.069103,93.986802,555.262035,60207.521688,1.300868e+07,1.408288,1.408288,0.341818,31120.828652
503,3.754236e+08,0.000000e+00,0.000000e+00,3.754236e+08,1.681702e+06,2401.364815,29455.863539,6674.201540,350.535814,227.124441,1376.476997,148093.095857,3.434943e+07,4.604239,4.563272,1.216497,71271.052711
601,9.396342e+07,0.000000e+00,0.000000e+00,9.396342e+07,3.762930e+05,280.764983,677.054433,877.296902,68.583581,46.125279,272.424718,27829.444432,6.071306e+06,0.012890,0.012890,0.003129,14973.169195
602,4.924693e+08,0.000000e+00,0.000000e+00,4.924693e+08,2.006008e+06,2483.486702,47957.684765,5730.546471,383.154966,252.201712,1492.438214,180715.971671,3.885003e+07,7.687900,7.687900,1.865995,82808.660169
603,3.071488e+08,0.000000e+00,0.000000e+00,3.071488e+08,1.390258e+06,2740.005501,27466.605658,8493.631791,290.383408,193.173871,1183.657912,130770.235617,3.080139e+07,4.175810,4.132659,1.144196,57169.859717
604,3.960850e+08,0.000000e+00,0.000000e+00,3.960850e+08,1.929248e+06,3776.820171,39118.708610,11195.027177,433.766383,280.007039,1729.145419,181897.600229,4.380663e+07,6.176449,6.085977,1.743290,83118.436910
605,4.649546e+08,0.000000e+00,0.000000e+00,4.649546e+08,2.099956e+06,3432.053909,61381.004423,8448.638533,455.533079,290.521095,1760.425170,203223.084829,4.659046e+07,9.985490,9.941486,2.538805,92183.633795
606,4.230277e+08,3.522147e+05,6.215554e+04,4.226134e+08,1.866135e+06,2708.965536,55517.807759,6039.935860,378.943683,242.942678,1457.825102,175094.563608,3.923362e+07,9.120064,9.082436,2.273573,79359.721261


# Calculate Post-BMP Loads with Attenuation

In [293]:
post_bmp_load = post_bmp_no_att[['Runoff','Effluent Volume','Volume Reduction','Untreated Volume']]
for p in pollutants:
    p_removal = att_removal[[p]]
    _load = (post_bmp_no_att
             .reset_index()
             .join(p_removal, on = 'HUC')
             .assign(Load = lambda df: df[p+' Load']*(1-df[p]))
             .set_index(['HUC','Land Use'])
             .rename(columns = {'Load':p+' Attenuated Load'})
             [[p+' Attenuated Load']]
            )
    post_bmp_load = post_bmp_load.join(_load)
post_bmp_load.tail(10)

Runoff  Effluent Volume  Volume Reduction  \
HUC Land Use                                                    
606 OPS       1.439002e+07     0.000000e+00      0.000000e+00   
    RES       2.009169e+07     0.000000e+00      0.000000e+00   
    TRA       9.255939e+06     0.000000e+00      0.000000e+00   
607 AGR       4.674467e+08     0.000000e+00      0.000000e+00   
    COM       1.868499e+08     7.147007e+07      1.261237e+07   
    FOR       6.673477e+07     0.000000e+00      0.000000e+00   
    OPS       5.775720e+07     0.000000e+00      0.000000e+00   
    PUB       5.641522e+06     0.000000e+00      0.000000e+00   
    RES       3.246312e+08     0.000000e+00      0.000000e+00   
    TRA       6.115632e+07     0.000000e+00      0.000000e+00   

              Untreated Volume  TSS Attenuated Load  TP Attenuated Load  \
HUC Land Use                                                              
606 OPS           1.439002e+07         4.752241e+04          157.210250   
    RES           2.009169e+07         1.696601e+05          512.111101   
    TRA           9.255939e+06         8.720826e+04          200.676426   
607 AGR           4.674467e+08         1.926006e+06         2392.915962   
    COM           1.027674e+08         5.603218e+05         4199.588932   
    FOR           6.673477e+07         2.749651e+05           66.658210   
    OPS           5.775720e+07         1.907407e+05          630.994611   
    PUB           5.641522e+06         2.790742e+04           96.436724   
    RES           3.246312e+08         2.741282e+06         8274.429754   
    TRA           6.115632e+07         5.762069e+05         1325.919551   

              NO3 Attenuated Load  NH3 Attenuated Load  PB Attenuated Load  \
HUC Land Use                                                                 
606 OPS                359.337714           662.528911            2.355658   
    RES                848.527115          1845.167948           26.748683   
    TRA                884.083787           990.742044           32.016367   
607 AGR              71349.750320          3326.736825          391.037486   
    COM               5396.789825         10155.516733          264.248746   
    FOR                 95.821177             8.332276           55.826251   
    OPS               1442.273396          2659.191574            9.454903   
    PUB                177.247942           345.505723            8.509122   
    RES              13710.067989         29813.281827          432.191563   
    TRA               5841.363629          6546.081517          211.540181   

              CU Attenuated Load  ZN Attenuated Load  BOD Attenuated Load  \
HUC Land Use                                                                
606 OPS                 3.773046           22.358791          4192.273333   
    RES                18.856289          126.595141         12244.258512   
    TRA                18.490641          122.153799          8588.692762   
607 AGR               248.046167         1459.095099        188806.905755   
    COM               211.973070         1150.517195         98008.043604   
    FOR                35.412174          208.306905         19443.366555   
    OPS                15.143871           89.741456         16826.522952   
    PUB                 4.499925           26.512013          2915.091414   
    RES               304.670299         2045.459661        197836.478915   
    TRA               122.172311          807.100831         56747.650217   

              E.COLI Attenuated Load  GLYPHOSATE Attenuated Load  \
HUC Land Use                                                       
606 OPS                 8.983443e+05                    0.000000   
    RES                 3.670052e+06                    0.125429   
    TRA                 3.468151e+06                    0.000000   
607 AGR                 3.910375e+07                   12.022944   
    COM                 3.169693e+07                    0.698950   


In [296]:
post_bmp_load.to_csv(os.path.join(result_path,'Post-BMP_Load_WithAttenuation.csv'))

post_bmp_load_total = post_bmp_load.reset_index().groupby(['HUC']).sum()
post_bmp_load_total.to_csv(os.path.join(result_path,'Post-BMP_Load_WithAttenuation_SubbasinTotals.csv'))
post_bmp_load_total.tail(10)

,Runoff,Effluent Volume,Volume Reduction,Untreated Volume,TSS Attenuated Load,TP Attenuated Load,NO3 Attenuated Load,NH3 Attenuated Load,PB Attenuated Load,CU Attenuated Load,ZN Attenuated Load,BOD Attenuated Load,E.COLI Attenuated Load,GLYPHOSATE Attenuated Load,"2,4-D Attenuated Load",CARBARYL Attenuated Load,OIL&GREASE Attenuated Load
HUC,,,,,,,,,,,,,,,,,
501,1.311734e+08,0.000000e+00,0.000000e+00,1.311734e+08,5.417455e+05,170.467634,310.969425,206.104188,109.657812,69.848284,411.836526,38628.926454,8.390665e+06,0.000000,0.000000,0.000000,23569.868230
502,1.821886e+08,0.000000e+00,0.000000e+00,1.821886e+08,7.437310e+05,620.485679,9113.425690,1407.465789,144.069103,93.986802,555.262035,60207.521688,1.300868e+07,1.408288,1.408288,0.341818,31120.828652
503,3.754236e+08,0.000000e+00,0.000000e+00,3.754236e+08,1.681702e+06,2401.364815,29455.863539,6674.201540,350.535814,227.124441,1376.476997,148093.095857,3.434943e+07,4.604239,4.563272,1.216497,71271.052711
601,9.396342e+07,0.000000e+00,0.000000e+00,9.396342e+07,3.762930e+05,280.764983,677.054433,877.296902,68.583581,46.125279,272.424718,27829.444432,6.071306e+06,0.012890,0.012890,0.003129,14973.169195
602,4.924693e+08,0.000000e+00,0.000000e+00,4.924693e+08,2.006008e+06,2483.486702,47957.684765,5730.546471,383.154966,252.201712,1492.438214,180715.971671,3.885003e+07,7.687900,7.687900,1.865995,82808.660169
603,3.071488e+08,0.000000e+00,0.000000e+00,3.071488e+08,8.817155e+05,2038.418873,20433.698880,6318.811890,221.340833,147.244176,1053.766844,116419.902271,2.742133e+07,2.953835,3.679154,0.809368,39394.521179
604,3.960850e+08,0.000000e+00,0.000000e+00,3.960850e+08,1.929248e+06,3776.820171,39118.708610,11195.027177,433.766383,280.007039,1729.145419,181897.600229,4.380663e+07,6.176449,6.085977,1.743290,83118.436910
605,4.649546e+08,0.000000e+00,0.000000e+00,4.649546e+08,2.099956e+06,3432.053909,61381.004423,8448.638533,455.533079,290.521095,1760.425170,203223.084829,4.659046e+07,9.985490,9.941486,2.538805,92183.633795
606,4.230277e+08,3.522147e+05,6.215554e+04,4.226134e+08,1.866135e+06,2708.965536,55517.807759,6039.935860,378.943683,242.942678,1457.825102,175094.563608,3.923362e+07,9.120064,9.082436,2.273573,79359.721261


# Calculate Load Reduction

In [318]:
load_reduction = (pre_bmp_load.drop(['Runoff'], axis = 1) 
                  - post_bmp_load.drop(['Runoff','Effluent Volume','Volume Reduction','Untreated Volume'], axis = 1)
                 )
load_reduction.to_csv(os.path.join(result_path,'Load_Reduction.csv'))
load_reduction.tail(10)

TSS Attenuated Load  TP Attenuated Load  NO3 Attenuated Load  \
HUC Land Use                                                                 
606 OPS                  0.000000            0.000000             0.000000   
    RES                  0.000000            0.000000             0.000000   
    TRA                  0.000000            0.000000             0.000000   
607 AGR                  0.000000            0.000000             0.000000   
    COM             392654.498566         1059.939904          2545.674305   
    FOR                  0.000000            0.000000             0.000000   
    OPS                  0.000000            0.000000             0.000000   
    PUB                  0.000000            0.000000             0.000000   
    RES                  0.000000            0.000000             0.000000   
    TRA                  0.000000            0.000000             0.000000   

              NH3 Attenuated Load  PB Attenuated Load  CU Attenuated Load  \
HUC Land Use                                                                
606 OPS                  0.000000            0.000000            0.000000   
    RES                  0.000000            0.000000            0.000000   
    TRA                  0.000000            0.000000            0.000000   
607 AGR                  0.000000            0.000000            0.000000   
    COM               8051.088577          207.280009           90.687236   
    FOR                  0.000000            0.000000            0.000000   
    OPS                  0.000000            0.000000            0.000000   
    PUB                  0.000000            0.000000            0.000000   
    RES                  0.000000            0.000000            0.000000   
    TRA                  0.000000            0.000000            0.000000   

              ZN Attenuated Load  BOD Attenuated Load  E.COLI Attenuated Load  \
HUC Land Use                                                                    
606 OPS                 0.000000              0.00000            0.000000e+00   
    RES                 0.000000              0.00000            0.000000e+00   
    TRA                 0.000000              0.00000            0.000000e+00   
607 AGR                 0.000000              0.00000            0.000000e+00   
    COM               778.275382          60086.16446            6.178415e+06   
    FOR                 0.000000              0.00000            0.000000e+00   
    OPS                 0.000000              0.00000            0.000000e+00   
    PUB                 0.000000              0.00000            0.000000e+00   
    RES                 0.000000              0.00000            0.000000e+00   
    TRA                 0.000000              0.00000            0.000000e+00   

              GLYPHOSATE Attenuated Load  2,4-D Attenuated Load  \
HUC Land Use                                                      
606 OPS                          0.00000                0.00000   
    RES                          0.00000                0.00000   
    TRA                          0.00000                0.00000   
607 AGR                          0.00000                0.00000   
    COM                          0.11758                0.11758   
    FOR                          0.00000                0.00000   
    OPS                          0.00000                0.00000   
    PUB                          0.00000                0.00000   
    RES                          0.00000                0.00000   
    TRA                          0.00000                0.00000   

              CARBARYL Attenuated Load  OIL&GREASE Attenuated Load  
HUC Land Use                                                        
606 OPS                        0.00000                    0.000000  
    RES                        0.00000                    0.000000  
    TRA                        0.00000                    0.000000  
607 AGR                      

In [325]:
load_reduction_total = load_reduction.reset_index().groupby(['HUC']).sum()
load_reduction_total.to_csv(os.path.join(result_path,'Load_Reduction_SubbasinTotals.csv'))
load_reduction_total.tail(10)

,TSS Attenuated Load,TP Attenuated Load,NO3 Attenuated Load,NH3 Attenuated Load,PB Attenuated Load,CU Attenuated Load,ZN Attenuated Load,BOD Attenuated Load,E.COLI Attenuated Load,GLYPHOSATE Attenuated Load,"2,4-D Attenuated Load",CARBARYL Attenuated Load,OIL&GREASE Attenuated Load
HUC,,,,,,,,,,,,,
501,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
502,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
503,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
601,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
602,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
603,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
604,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
605,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
606,1935.057494,5.223535,12.545447,39.676915,1.021506,0.446920,3.835452,296.113207,3.044811e+04,0.000579,0.000579,0.000579,85.495029


In [326]:
load_reduction_percent = load_reduction / pre_bmp_load.drop(['Runoff'], axis = 1) * 100
load_reduction_percent.to_csv(os.path.join(result_path,'Load_Reduction_Percent.csv'))
load_reduction_percent.tail(10)

TSS Attenuated Load  TP Attenuated Load  NO3 Attenuated Load  \
HUC Land Use                                                                 
606 OPS                  0.000000            0.000000             0.000000   
    RES                  0.000000            0.000000             0.000000   
    TRA                  0.000000            0.000000             0.000000   
607 AGR                  0.000000            0.000000             0.000000   
    COM                 41.202966           20.152754            32.051442   
    FOR                  0.000000            0.000000             0.000000   
    OPS                  0.000000            0.000000             0.000000   
    PUB                  0.000000            0.000000             0.000000   
    RES                  0.000000            0.000000             0.000000   
    TRA                  0.000000            0.000000             0.000000   

              NH3 Attenuated Load  PB Attenuated Load  CU Attenuated Load  \
HUC Land Use                                                                
606 OPS                  0.000000            0.000000            0.000000   
    RES                  0.000000            0.000000            0.000000   
    TRA                  0.000000            0.000000            0.000000   
607 AGR                  0.000000            0.000000            0.000000   
    COM                 44.220701           43.959145           29.963373   
    FOR                  0.000000            0.000000            0.000000   
    OPS                  0.000000            0.000000            0.000000   
    PUB                  0.000000            0.000000            0.000000   
    RES                  0.000000            0.000000            0.000000   
    TRA                  0.000000            0.000000            0.000000   

              ZN Attenuated Load  BOD Attenuated Load  E.COLI Attenuated Load  \
HUC Land Use                                                                    
606 OPS                 0.000000             0.000000                  0.0000   
    RES                 0.000000             0.000000                  0.0000   
    TRA                 0.000000             0.000000                  0.0000   
607 AGR                 0.000000             0.000000                  0.0000   
    COM                40.350393            38.006556                 16.3125   
    FOR                 0.000000             0.000000                  0.0000   
    OPS                 0.000000             0.000000                  0.0000   
    PUB                 0.000000             0.000000                  0.0000   
    RES                 0.000000             0.000000                  0.0000   
    TRA                 0.000000             0.000000                  0.0000   

              GLYPHOSATE Attenuated Load  2,4-D Attenuated Load  \
HUC Land Use                                                      
606 OPS                              NaN                    NaN   
    RES                              0.0                    0.0   
    TRA                              NaN                    NaN   
607 AGR                              0.0                    0.0   
    COM                             14.4                   14.4   
    FOR                              NaN                    NaN   
    OPS                              NaN                    NaN   
    PUB                              0.0                    0.0   
    RES                              0.0                    0.0   
    TRA                              NaN                    NaN   

              CARBARYL Attenuated Load  OIL&GREASE Attenuated Load  
HUC Land Use                                                        
606 OPS                            NaN                    0.000000  
    RES                            0.0                    0.000000  
    TRA                            NaN                    0.000000  
607 AGR                      

In [329]:
load_reduction_percent_total = load_reduction_total / pre_bmp_load_total.drop(['Runoff'], axis = 1) * 100
load_reduction_percent_total.to_csv(os.path.join(result_path,'Load_Reduction_Percent_SubbasinTotals.csv'))
load_reduction_percent_total.tail(10)

,TSS Attenuated Load,TP Attenuated Load,NO3 Attenuated Load,NH3 Attenuated Load,PB Attenuated Load,CU Attenuated Load,ZN Attenuated Load,BOD Attenuated Load,E.COLI Attenuated Load,GLYPHOSATE Attenuated Load,"2,4-D Attenuated Load",CARBARYL Attenuated Load,OIL&GREASE Attenuated Load
HUC,,,,,,,,,,,,,
501,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000
502,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
503,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
601,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
602,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
603,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
604,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
605,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
606,0.103586,0.192453,0.022592,0.652622,0.268842,0.183623,0.262404,0.168831,0.077547,0.006353,0.006380,0.025480,0.107615


In [16]:
example = [{
                    "name":"Nutrient Management Plan - Agriculture",
                    "landuses":[{
                        "name":"AGR",
                        "pct_implemented": 0
                    }]
}]


In [17]:
for bmp in example:
    print(bmp)

{'name': 'Nutrient Management Plan - Agriculture', 'landuses': [{'name': 'AGR', 'pct_implemented': 0}]}


In [18]:
bmp['name']


'Nutrient Management Plan - Agriculture'

In [23]:
pd.DataFrame(index = land_uses, columns = bmps).loc['AGR','Conservation Cover']

nan